In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_msgpack('preprocess/Readmission/Split_Notes_Texts_combined.msg')

In [17]:
df.head()

,subject_id,hadm_id,icustay_id,cseq,y,exp_split
0,3,145834,211552,5 49 pm ct_abdomen_w o_contrast ct_pelvis_w o_...,0,train
1,4,185777,294638,4 20 pm chest portable_ap clip reason fever do...,0,train
2,6,107064,228232,4 25 pm abdomen supine_only in_o.r. clip reaso...,0,train
3,11,194540,229441,5 01 am mr_head_w w o contrast clip reason eva...,0,train
4,12,112213,232669,vent settings changed to vt 600 simv 16 fio 2 ...,1,train


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import re
from tqdm import tqdm_notebook

pattern = re.compile(r'\s\W+\s')
texts = list(df['cseq'])
for t in tqdm_notebook(range(len(texts))) :
    texts[t] = re.sub(r'\s+', ' ', pattern.sub(' ', texts[t]))

In [5]:
df['cseq'] = texts

In [6]:
tokenizer = lambda s : s.split(' ')
vocab = CountVectorizer(max_features=30000, tokenizer=tokenizer, stop_words='english')
vocab.fit(list(df[df['exp_split'] == 'train']['cseq']))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7fbf99098268>, vocabulary=None)

In [7]:
bow = {}
for key in ['train', 'dev'] :
    bow[key] = vocab.transform(list(df[df['exp_split'] == key]['cseq']))

In [8]:
label = {}
for key in ['train', 'dev'] :
    label[key] = list(df[df['exp_split'] == key]['y'])

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [10]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(bow['train'].toarray(), label['train'])
predict = lr.predict_proba(bow['dev'].toarray())
print(roc_auc_score(label['dev'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6906116955761423


In [11]:
from sklearn.preprocessing import normalize
normalised_bow = {}
for key in ['train', 'dev'] :
    normalised_bow[key] = normalize(bow[key], norm='l2', copy=True)

In [12]:
lr_norm = LogisticRegression(class_weight='balanced')
lr_norm.fit(normalised_bow['train'].toarray(), label['train'])
predict = lr_norm.predict_proba(normalised_bow['dev'].toarray())
print(roc_auc_score(label['dev'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7348514899492778


In [13]:
binary_bow = {}
for key in ['train', 'dev'] :
    binary_bow[key] = np.clip(bow[key].toarray(), 0, 1)

In [14]:
lr_bin = LogisticRegression(class_weight='balanced')
lr_bin.fit(binary_bow['train'], label['train'])
predict = lr_bin.predict_proba(binary_bow['dev'])
print(roc_auc_score(label['dev'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6282727070492112


In [15]:
bin_normalised_bow = {}
for key in ['train', 'dev'] :
    bin_normalised_bow[key] = normalize(binary_bow[key], norm='l2', copy=True)

In [16]:
lr_bin = LogisticRegression(class_weight='balanced')
lr_bin.fit(bin_normalised_bow['train'], label['train'])
predict = lr_bin.predict_proba(bin_normalised_bow['dev'])
print(roc_auc_score(label['dev'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7321257010920454
